# Simulate Trading Bot Action

<div style="display: flex;">
    <div style="flex: 1; display: flex; align-items: center;">
        <img src="src/A.png" style="width: 100%;">
    </div>
    <div style="flex: 1; display: flex; align-items: center;">
        <img src="src/B.png" style="width: 100%;">
    </div>
</div>

## Download Historical Data

### Download 100 prior days from today

In [1]:
from datetime import datetime, timedelta

N_DAYS = 100

date_end = datetime.today().date()
date_start = date_end - timedelta(days=N_DAYS)
date_start = str(date_start)
date_start

'2024-02-29'

### Import Library

In [2]:
import sys
sys.path.append('../../../modules/')

In [3]:
import utils

### Download Data

In [4]:
symbol = 'BTC/USD'
data = utils.get_data_crypto_daily(symbol=symbol, start=date_start)
data

{'bars': {'BTC/USD': [{'c': 61335.7055,
    'h': 63654.5755,
    'l': 60390.816,
    'n': 510,
    'o': 62504.9235,
    't': '2024-02-29T06:00:00Z',
    'v': 6.677049422,
    'vw': 62004.0155700017},
   {'c': 62079.7935,
    'h': 63110.2535,
    'l': 61219.93,
    'n': 221,
    'o': 61295.611,
    't': '2024-03-01T06:00:00Z',
    'v': 6.626800627,
    'vw': 61998.4901195568},
   {'c': 61943.201,
    'h': 62344.33,
    'l': 61682.798,
    'n': 131,
    'o': 62076.62,
    't': '2024-03-02T06:00:00Z',
    'v': 0.840388591,
    'vw': 62031.3562011778},
   {'c': 63395.746,
    'h': 64265.162,
    'l': 61453.42,
    'n': 100,
    'o': 61934.7,
    't': '2024-03-03T06:00:00Z',
    'v': 1.675955527,
    'vw': 62617.1338362145},
   {'c': 67002.88,
    'h': 68828.19,
    'l': 63027.295285359,
    'n': 689,
    'o': 63392.359,
    't': '2024-03-04T06:00:00Z',
    'v': 11.111514001,
    'vw': 66647.1650367126},
   {'c': 66418.9025,
    'h': 70025.188916876,
    'l': 59553.34987593,
    'n': 799,
 

### Process the DataFrame

In [5]:
df = utils.process_df(data=data, symbol=symbol)
df

,Open,High,Low,Close,Volume
Date,,,,,
2024-02-29 06:00:00+00:00,62504.9235,63654.575500,60390.816000,61335.7055,6.677049
2024-03-01 06:00:00+00:00,61295.6110,63110.253500,61219.930000,62079.7935,6.626801
2024-03-02 06:00:00+00:00,62076.6200,62344.330000,61682.798000,61943.2010,0.840389
2024-03-03 06:00:00+00:00,61934.7000,64265.162000,61453.420000,63395.7460,1.675956
2024-03-04 06:00:00+00:00,63392.3590,68828.190000,63027.295285,67002.8800,11.111514
...,...,...,...,...,...
2024-06-04 05:00:00+00:00,69006.0950,71257.410000,68564.163500,71226.0910,6.400314
2024-06-05 05:00:00+00:00,71146.9500,71736.967632,70370.321000,71101.4380,1.957128
2024-06-06 05:00:00+00:00,71059.9955,71622.498000,70125.820000,71202.7300,1.823753


## Bollinger Strategy

### Calculate Bands

In [6]:
df_bollinger = df[['Close']].copy()

k = 2
n_windows = 30

rolling_std = df_bollinger.Close.rolling(n_windows).std()
rolling_mean = df_bollinger.Close.rolling(n_windows).mean()

df_bollinger = (df_bollinger
 .assign(
     upper = rolling_mean + rolling_std * k,
     lower = rolling_mean - rolling_std * k)
)

df_bollinger

,Close,upper,lower
Date,,,
2024-02-29 06:00:00+00:00,61335.7055,NaN,NaN
2024-03-01 06:00:00+00:00,62079.7935,NaN,NaN
2024-03-02 06:00:00+00:00,61943.2010,NaN,NaN
2024-03-03 06:00:00+00:00,63395.7460,NaN,NaN
2024-03-04 06:00:00+00:00,67002.8800,NaN,NaN
...,...,...,...
2024-06-04 05:00:00+00:00,71226.0910,72819.293037,59952.361796
2024-06-05 05:00:00+00:00,71101.4380,73198.338464,60083.158903
2024-06-06 05:00:00+00:00,71202.7300,73501.029016,60359.970351


### Visualize Data

In [7]:
df_bollinger.plot();

## Prepare Alpaca Trading API

In [8]:
import os

API_KEY = os.getenv('API_KEY')
API_SECRET = os.getenv('API_SECRET')

In [9]:
from alpaca.trading.client import TradingClient

client = TradingClient(API_KEY, API_SECRET, paper=True)

## Configure market order request

In [10]:
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

symbol = 'BTCUSD'

request_order = MarketOrderRequest(
    symbol=symbol,
    qty=0.1,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.GTC)

request_order

{   'client_order_id': None,
    'extended_hours': None,
    'notional': None,
    'order_class': None,
    'position_intent': None,
    'qty': 0.1,
    'side': <OrderSide.BUY: 'buy'>,
    'stop_loss': None,
    'symbol': 'BTCUSD',
    'take_profit': None,
    'time_in_force': <TimeInForce.GTC: 'gtc'>,
    'type': <OrderType.MARKET: 'market'>}

```python
# Order to Buy and Open Position
request = client_trading.submit_order(order_data=request_order)

# Close position
request = client_trading.close_position(symbol)
```

## Signals to buy, sell or pass

### Get last price

In [11]:
last_day = df_bollinger.iloc[-1]
last_day

Close    69431.839000
upper    73537.156804
lower    61280.360163
Name: 2024-06-08 05:00:00+00:00, dtype: float64

In [12]:
close, upper, lower = last_day

### Open position?

In [13]:
try:
    client.get_open_position(symbol)
    flag_bought = True
except Exception as e:
    flag_bought = False
    print(e)

### Conditionals

In [14]:
if (lower > close) and  (not flag_bought):
    request = client.submit_order(order_data=request_order)
elif (close > upper) and (flag_bought):
    request = client.close_position(symbol)
else:
    request = 'No trade executed.'
    
request

'No trade executed.'